# Prepare

In [1]:
#@markdown ## Save Mode

saveMode = "temporary" #@param ["temporary", "googleDrive"] {"type": "string"}
savePath = ""
if saveMode.lower() == "temporary":
  savePath = "/content/"
else:
  savePath = "/content/drive/MyDrive/"

In [ ]:
#@markdown ## Mount to drive
#@markdown ### If you use googleDrive mode run this cell
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
#@markdown ## Install requirements
!pip install -U pyngrok

In [ ]:
#@markdown ## Create Folder
import os, os.path
cf = input("Area you sure? [Y/N]: ")
if cf.lower() == "y":
  if os.path.exists(savePath + "TerrariaServer"):
    print("Folder exists!")
  else:
    print("Creating Folder...")
    os.system(f"mkdir " + savePath + "TerrariaServer")
    print("Done!")
  if os.path.exists(savePath + "/TerrariaServer/Worlds"):
    print("Folder exists!")
  else:
    print("Creating Folder...")
    os.system(f"mkdir " + savePath + "TerrariaServer/Worlds")
    print("Done!")

In [ ]:
#@markdown ## Ngrok auth token (For tunneling the server) [Required]
import getpass
print("Get auth token from https://dashboard.ngrok.com/get-started/your-authtoken")
ngtoken = getpass.getpass()
with open(savePath+"TerrariaServer/ngtoken.txt", "w") as f:
  f.write(ngtoken)

# Setup Server

In [ ]:
#@markdown # Download Server
serverUrl = "https://terraria.org/api/download/pc-dedicated-server/terraria-server-1449.zip" #@param {type: "string"}
!wget $serverUrl -O terraria-server.zip
!unzip /content/terraria-server.zip -d /content/TerrariaServer

In [6]:
#@markdown # Create Config

with open(savePath+"TerrariaServer/terraria.properties", "w") as f:
  f.write("""
#this is an example config file for TerrariaServer.exe
#please report crashes by emailing crashlog.txt to support@terraria.org

#the following is a list of available command line parameters:

#-config <config file>				            Specifies the configuration file to use.
#-port <port number>				              Specifies the port to listen on.
#-players <number> / -maxplayers <number>	Sets the max number of players
#-pass <password> / -password <password>	Sets the server password
#-world <world file>				              Load a world and automatically start the server.
#-autocreate <#>			                  	Creates a world if none is found in the path specified by -world. World size is specified by: 1(small), 2(medium), and 3(large).
#-banlist <path>			                  	Specifies the location of the banlist. Defaults to "banlist.txt" in the working directory.
#-worldname <world name>             			Sets the name of the world when using -autocreate.
#-secure			                        		Adds addition cheat protection to the server.
#-noupnp				                        	Disables automatic port forwarding
#-steam                         					Enables Steam Support
#-lobby <friends> or <private>             Allows friends to join the server or sets it to private if Steam is enabled
#-ip <ip address>	Sets the IP address for the server to listen on
#-forcepriority <priority>	Sets the process priority for this task. If this is used the "priority" setting below will be ignored.
#-disableannouncementbox   Disables the text announcements Announcement Box makes when pulsed from wire.
#-announcementboxrange <number>   Sets the announcement box text messaging range in pixels, -1 for serverwide announcements.
#-seed <seed> Specifies the world seed when using -autocreate

#remove the # in front of commands to enable them.

port=7777 #Don't Change

#Load a world and automatically start the server.
world=/content/TerrariaServer/Worlds/world.wld

#Creates a new world if none is found. World size is specified by: 1(small), 2(medium), and 3(large).
autocreate=1

#Sets the world seed when using autocreate
#seed=AwesomeSeed

#Sets the name of the world when using autocreate
worldname=world

#Sets the difficulty of the world when using autocreate 0(normal), 1(expert)
#difficulty=1

#Sets the max number of players allowed on a server.  Value must be between 1 and 255
maxplayers=10

#Set the server password
#password=setyourpassword

#Set the message of the day
motd=Server.pro | MMB LMAOOO

#Sets the folder where world files will be stored
worldpath=/content/TerrariaServer/Worlds

#The location of the banlist. Defaults to "banlist.txt" in the working directory.
banlist=banlist.txt

#Adds addition cheat protection.
#secure=1

#Sets the server language from its language code.
#English = en-US, German = de-DE, Italian = it-IT, French = fr-FR, Spanish = es-ES, Russian = ru-RU, Chinese = zh-Hans, Portuguese = pt-BR, Polish = pl-PL,
#language=en-US

#Reduces enemy skipping but increases bandwidth usage. The lower the number the less skipping will happen, but more data is sent. 0 is off.
#npcstream=60
""")

with open(savePath+"TerrariaServer/banlist.txt", "w") as f:
  f.write("")
print("Done!")

Done!


In [ ]:
#@markdown # Start Server
%cd "$savePath"TerrariaServer/1449/Linux
!sudo chmod +x TerrariaServer.bin.x86_64*
import pyngrok
from pyngrok import ngrok, conf

serverRegion = "ap" #@param ["us", "eu", "ap", "au", "sa", "jp", "in"] {type: "string"}
conf.get_default().region = serverRegion

with open(savePath+"TerrariaServer/ngtoken.txt", "r") as f:
  auth_token = f.read().strip()
  if auth_token:
    !ngrok authtoken "$auth_token"
  else:
    raise ValueError("Failed to get ngrok token!")

client = ngrok.connect("7777", 'tcp')
serverip = ((str(client).split('"')[1::2])[0]).replace('tcp://', '')
print("Server IP: " + serverip)

import subprocess as sub
import threading

server = sub.Popen(["./TerrariaServer.bin.x86_64", "-config", "/content/TerrariaServer/terraria.properties"], stdin=sub.PIPE, stdout=sub.PIPE, stderr=sub.PIPE,text=True)
def printLog():
  global server
  prevOut = ""
  while True:
    out = server.stdout.readline()
    if out == '' and server.poll() is not None:
      break
    if out != prevOut:
      print(out.strip())
      prevOut = out

threading.Thread(target=printLog).start()
while True:
  try:
    cmd = input()
    cmd2 = cmd.lower()
    if cmd2 == "stop" or cmd2 == "exit" or cmd2 == "exit-nosave":
      if cmd2 == "exit-nosave":
        server.stdin.write("exit-nosave\n")
      else:
        server.stdin.write("exit\n")
      server.stdin.flush()
      break
    else:
      server.stdin.write(cmd+"\n")
      server.stdin.flush()
  except KeyboardInterrupt:
    server.stdin.write("exit\n")
    server.stdin.flush()
    break

#!./TerrariaServer.bin.x86_64 -config /content/TerrariaServer/terraria.properties